In [ ]:
from matplotlib import pyplot as plt
import numpy as np
from IPython.display import clear_output
import visdom


class IPlot(callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.acc = []
        self.val_acc = []
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.acc.append(logs.get('categorical_accuracy'))
        self.val_acc.append(logs.get('val_categorical_accuracy'))
        self.i += 1
        f, (ax1, ax2) = plt.subplots(1, 2, sharex=True, figsize=(15,5))
        
        clear_output(wait=True)
        
        ax1.set_yscale('log')
        ax1.plot(self.x, self.losses, label="loss")
        ax1.plot(self.x, self.val_losses, label="val_loss")
        ax1.legend()
        
        ax2.plot(self.x, self.acc, label="acc")
        ax2.plot(self.x, self.val_acc, label="val_acc")
        ax2.legend()
        
        plt.show();
        
        
class VisPlot(callbacks.Callback):
    
    def on_train_begin(self, logs={}):
        
        self.vis = visdom.Visdom()
        self.win_loss = None
        self.win_acc = None
    
    def on_epoch_end(self, epoch, logs={}):
        
        x = np.array([epoch])
        y_loss = np.column_stack((np.array(logs.get('loss')), np.array(logs.get('val_loss'))))
        y_acc = np.column_stack((np.array(logs.get('categorical_accuracy')),
                                        np.array(logs.get('val_categorical_accuracy'))))
        legend_loss = dict(legend=['train_loss', 'val_loss'])
        legend_acc = dict(legend=['train_acc', 'val_acc'])
        
        if epoch == 0:
            self.win_loss = self.vis.line(Y=y_loss,
                                          X=x,
                                          opts=legend_loss)
            self.win_acc = self.vis.line(Y=y_acc,
                                         X=x,
                                         opts=legend_acc)
        else:
            self.vis.line(win=self.win_loss, 
                     update='append',
                     Y=y_loss,
                     X=x,
                     opts=legend_loss)
            self.vis.line(win=self.win_acc, 
                     update='append',
                     Y=y_acc,
                     X=x,
                     opts=legend_acc)
    


        
visplot = VisPlot()